In [25]:
import numpy as np
from numpy.random import default_rng
from qiskit import QuantumCircuit

# Conjugation of local Paulis by CZ (up to global phase)
CZ_PAULI_CONJ = {
    ('I', 'I'): ('I', 'I'),
    ('I', 'X'): ('Z', 'X'),
    ('I', 'Y'): ('Z', 'Y'),
    ('I', 'Z'): ('I', 'Z'),

    ('X', 'I'): ('X', 'Z'),
    ('X', 'X'): ('Y', 'Y'),
    ('X', 'Y'): ('Y', 'X'),
    ('X', 'Z'): ('X', 'I'),

    ('Y', 'I'): ('Y', 'Z'),
    ('Y', 'X'): ('X', 'Y'),
    ('Y', 'Y'): ('X', 'X'),
    ('Y', 'Z'): ('Y', 'I'),

    ('Z', 'I'): ('Z', 'I'),
    ('Z', 'X'): ('I', 'X'),
    ('Z', 'Y'): ('I', 'Y'),
    ('Z', 'Z'): ('Z', 'Z'),
}

PAULIS = ['I', 'X', 'Y', 'Z']


def apply_single_pauli(qc: QuantumCircuit, qubit, label: str):
    """Apply Pauli {I,X,Y,Z} on `qubit` using only x, sx, rz (up to global phase)."""
    if label == 'I':
        return
    elif label == 'X':
        qc.x(qubit)
    elif label == 'Z':
        qc.rz(np.pi, qubit)  # Z ≃ Rz(pi)
    elif label == 'Y':
        # Y ≃ X·Rz(pi) up to global phase
        qc.x(qubit)
        qc.rz(np.pi, qubit)
    else:
        raise ValueError(f"Unknown Pauli label {label}")


def twirl_cz_gates_once(circ: QuantumCircuit, rng=None) -> QuantumCircuit:
    """
    Take an already-transpiled circuit and insert Pauli twirls around every CZ gate.
    """
    if rng is None:
        rng = default_rng()

    new_circ = QuantumCircuit(*circ.qregs, *circ.cregs, name=circ.name)
    new_circ.global_phase = circ.global_phase

    for inst, qargs, cargs in circ.data:
        if inst.name == "cz":
            q0, q1 = qargs

            # random local Paulis
            P1 = rng.choice(PAULIS)
            P2 = rng.choice(PAULIS)

            # conjugated Paulis after CZ
            P1p, P2p = CZ_PAULI_CONJ[(P1, P2)]

            # before CZ
            apply_single_pauli(new_circ, q0, P1)
            apply_single_pauli(new_circ, q1, P2)

            # CZ itself
            new_circ.append(inst, [q0, q1], cargs)

            # after CZ
            apply_single_pauli(new_circ, q0, P1p)
            apply_single_pauli(new_circ, q1, P2p)
        else:
            new_circ.append(inst, qargs, cargs)

    return new_circ


def generate_twirled_circuits(base_circ: QuantumCircuit,
                              num_randomizations: int,
                              seed: int | None = None):
    rng = default_rng(seed)
    return [twirl_cz_gates_once(base_circ, rng=rng)
            for _ in range(num_randomizations)]

def count_gates(qc: QuantumCircuit):
    gate_count = { qubit: 0 for qubit in qc.qubits }
    for gate in qc.data:
        for qubit in gate.qubits:
            gate_count[qubit] += 1
    return gate_count

def remove_idle_wires(qc: QuantumCircuit):
    qc_out = qc.copy()
    gate_count = count_gates(qc_out)
    for qubit, count in gate_count.items():
        if count == 0:
            qc_out.qubits.remove(qubit)
    return qc_out


In [ ]:
import numpy as np
from collections import Counter

from qiskit import QuantumCircuit, transpile
from qiskit_ibm_runtime.fake_provider import FakeMarrakesh  # example fake backend
from qiskit_ibm_runtime import SamplerV2
from qiskit_aer import AerSimulator

# ---- High-level circuit (can be in any gates, we'll transpile later) ----

# def hadamard_via_basis(qc, q):
#     """H using only sx and rz (up to global phase)."""
#     qc.rz(np.pi, q)
#     qc.sx(q)
#     qc.rz(np.pi, q)

qc = QuantumCircuit(2, 2)
qc.h(0)
qc.cx(0,1)
# hadamard_via_basis(qc, 0)
# qc.cz(0, 1)
qc.measure(0, 0)
qc.measure(1, 1)

print("Original circuit:")
print(qc)

# ---- Choose FakeBackend and do the *main* transpile first ----

backend = FakeMarrakesh() #should use a backend with cz as the 2 qubit gate

# Map to backend coupling, basis, and do heavy optimizations
qc_phys = transpile(
    qc,
    backend=backend,
    optimization_level=3,
)

print("\nTranspiled physical circuit (before twirling):")
print(qc_phys)

# ---- Twirl the *physical* CZ gates ----

num_randomizations = 16
twirled_circs = generate_twirled_circuits(qc_phys, num_randomizations, seed=1234)

twirled_circs=[remove_idle_wires(circ) for circ in twirled_circs]


# Optional: very light re-transpile just to fix up tiny details / scheduling,
# but not to re-optimize away our Paulis.
twirled_circs= transpile(
    twirled_circs,
    backend=backend,
    optimization_level=0,   # important: keep this low (keep zero)
)

# ---- Run and aggregate counts ----

shots = 1024
# job = backend.run(twirled_circs_t, shots=shots)
ideal_backend=AerSimulator()
job = backend.run(twirled_circs, shots=shots)
result = job.result()
print(result)

total_counts = Counter()
for i in range(num_randomizations):
    counts = result.get_counts(i)
    total_counts.update(counts)

print("\nAggregated counts over all twirled circuits:")
for bitstring, cnt in sorted(total_counts.items()):
    print(bitstring, cnt)

Original circuit:
     ┌───┐     ┌─┐   
q_0: ┤ H ├──■──┤M├───
     └───┘┌─┴─┐└╥┘┌─┐
q_1: ─────┤ X ├─╫─┤M├
          └───┘ ║ └╥┘
c: 2/═══════════╩══╩═
                0  1 

Transpiled physical circuit (before twirling):
global phase: 3π/4
          ┌─────────┐┌────┐ ┌───────┐    ┌────┐┌─────────┐┌─┐
q_1 -> 13 ┤ Rz(π/2) ├┤ √X ├─┤ Rz(π) ├──■─┤ √X ├┤ Rz(π/2) ├┤M├
          ├─────────┤├────┤┌┴───────┴┐ │ └┬─┬─┘└─────────┘└╥┘
q_0 -> 14 ┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├─■──┤M├──────────────╫─
          └─────────┘└────┘└─────────┘    └╥┘              ║ 
     c: 2/═════════════════════════════════╩═══════════════╩═
                                           0               1 
global phase: 3π/4
      ┌─────────┐┌────┐ ┌───────┐ ┌───────┐   ┌───────┐┌────┐┌─────────┐┌─┐
q_13: ┤ Rz(π/2) ├┤ √X ├─┤ Rz(π) ├─┤ Rz(π) ├─■─┤ Rz(π) ├┤ √X ├┤ Rz(π/2) ├┤M├
      ├─────────┤├────┤┌┴───────┴┐├───────┤ │ ├───────┤└┬─┬─┘└─────────┘└╥┘
q_14: ┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├┤ Rz(π) ├─■─┤ Rz(π) ├─┤M├──────────────╫─
    

C:\Users\main\AppData\Local\Temp\ipykernel_46456\3765140428.py:57: DeprecationWarning: Treating CircuitInstruction as an iterable is deprecated legacy behavior since Qiskit 1.2, and will be removed in Qiskit 3.0. Instead, use the `operation`, `qubits` and `clbits` named attributes.
  for inst, qargs, cargs in circ.data:


Result(backend_name='aer_simulator', backend_version='0.17.1', job_id='3f70b2d0-06bf-4aa0-abe9-ab6b811148a3', success=True, results=[ExperimentResult(shots=1024, success=True, meas_level=2, data=ExperimentResultData(counts={'0x0': 505, '0x3': 514, '0x1': 4, '0x2': 1}), header={'creg_sizes': [['c', 2]], 'global_phase': 2.3561944901923475, 'memory_slots': 2, 'n_qubits': 156, 'name': 'circuit-73795', 'qreg_sizes': [['q', 156]], 'metadata': {}}, status=DONE, seed_simulator=1545675784, metadata={'batched_shots_optimization': False, 'required_memory_mb': 1, 'method': 'density_matrix', 'active_input_qubits': [13, 14], 'device': 'CPU', 'remapped_qubits': True, 'num_qubits': 2, 'num_clbits': 2, 'time_taken': 0.0041002, 'sample_measure_time': 0.0001971, 'input_qubit_map': [[13, 0], [14, 1]], 'max_memory_mb': 32530, 'measure_sampling': True, 'noise': 'superop', 'parallel_shots': 1, 'parallel_state_update': 16, 'runtime_parameter_bind': False, 'num_bind_params': 1, 'fusion': {'enabled': True, 'thr